In [1]:
#!pip install -r data/requirements.txt

In [2]:
import json
import numpy as np
import pandas as pd

from tqdm import tqdm

In [3]:
def write_to_json_file(input_obj, file_name):
    json_object = input_obj.to_json(orient="records")
 
    # Writing to sample.json
    with open(file_name, "w") as outfile:
        outfile.write(json_object)

# Read db_data.csv File

In [4]:
df = pd.read_csv("data/db_data.csv")

print(df.head)

df.rename(columns = {'PTID':'patient_id'}, inplace = True)

df = df.drop('AD_dx_in_5_yrs', axis=1)

df.insert(loc = 1, column = 'ad_probability', value = -1)

# Write full DF as JSON
write_to_json_file(df, "data/db_data_full.json")

#df.head()

<bound method NDFrame.head of            PTID Diagnosis_at_Baseline   Age  Gender  Years_of_Education  \
0    022_S_0004                  LMCI  67.5    Male                  10   
1    011_S_0005                    CN  73.7    Male                  16   
2    100_S_0006                  LMCI  80.4  Female                  13   
3    022_S_0014                    CN  78.5  Female                  12   
4    100_S_0015                    CN  80.8    Male                  18   
..          ...                   ...   ...     ...                 ...   
492  126_S_1187                  LMCI  76.4    Male                  12   
493  023_S_1190                    CN  76.5  Female                  13   
494  007_S_1206                    CN  72.9    Male                  14   
495  136_S_1227                  LMCI  64.6  Female                  12   
496  094_S_0692                    CN  76.7  Female                  19   

           Ethnicity   Race  APOE4  MMSE  AD_dx_in_5_yrs  ...  \
0   

# Filter Data and Convert to JSON

In [5]:
# Split DFs into objects
df_1 = df.iloc[:,0:61]
df_2 = df.iloc[:,61:117]
df_3 = df.iloc[:,117:]


# Save as JSON arrays
write_to_json_file(df_1, "data/db_data_converted_part_1.json")
write_to_json_file(df_2, "data/db_data_converted_part_2.json")
write_to_json_file(df_3, "data/db_data_converted_part_3.json")

print(df_1.head())
print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
print(df_2.head())
print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
print(df_3.head())
print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

   patient_id  ad_probability Diagnosis_at_Baseline   Age  Gender  \
0  022_S_0004              -1                  LMCI  67.5    Male   
1  011_S_0005              -1                    CN  73.7    Male   
2  100_S_0006              -1                  LMCI  80.4  Female   
3  022_S_0014              -1                    CN  78.5  Female   
4  100_S_0015              -1                    CN  80.8    Male   

   Years_of_Education        Ethnicity   Race  APOE4  MMSE  ...  \
0                  10      Hisp/Latino  White    0.0  27.0  ...   
1                  16  Not Hisp/Latino  White    0.0  29.0  ...   
2                  13  Not Hisp/Latino  White    0.0  25.0  ...   
3                  12      Hisp/Latino  White    0.0  29.0  ...   
4                  18  Not Hisp/Latino  White    1.0  29.0  ...   

   FreeSurfer.convexity..75..1017  FreeSurfer.convexity..median.1013  \
0                        0.055770                          -0.312247   
1                       -0.006824     

# Convert Three DFs to JSON Array

In [6]:
data = [] # This is a list

# Convert DF fragments into JSON array strings
json_str_1 = df_1.to_json(orient="records")
json_str_2 = df_2.to_json(orient="records")
json_str_3 = df_3.to_json(orient="records")

# Convert to arrays
json_1 = json.loads(json_str_1)
json_2 = json.loads(json_str_2)
json_3 = json.loads(json_str_3)

for i in range(len(df_1)):
    # Append list
    row = [json_1[i], json_2[i], json_3[i]]
    data.append(row)

#json_data = json.dumps(data) # Convert the dictionary to a JSON string
#write_to_json_file(json_data, "data/db_data_converted.json")

# Convert the dictionary to a JSON string
with open('data/db_data_converted.json', 'w') as fout:
    json.dump(data, fout)




# Convert DF to SQL FILE

In [7]:
def get_sql_datatype(type_str):
    if type_str == "int64":
        return "INT"
    elif type_str == "float64":
        return "FLOAT"
    else:
        return "VARCHAR(20)"

sql_script_str = \
"""GRANT ALL PRIVILEGES ON DATABASE alzheimer_predict TO alzheimer_predict_user;
CREATE TABLE IF NOT EXISTS patients(id INT GENERATED ALWAYS AS IDENTITY,patient_id VARCHAR NOT NULL,created_at TIMESTAMP NOT NULL DEFAULT current_timestamp);
"""

# Add create records table query
create_records_table_query = """CREATE TABLE IF NOT EXISTS records(id INT GENERATED ALWAYS AS IDENTITY, """

n_cols = len(df.columns)
n_rows = len(df)

for i in range(n_cols):
    col_name = df.columns[i].replace(".", "")
    col_type = get_sql_datatype(str(df.dtypes[i]))
    create_records_table_query = create_records_table_query + f"{col_name} {col_type}, "

# Remove extra ", " and finish the query
create_records_table_query = create_records_table_query + """created_at TIMESTAMP NOT NULL DEFAULT current_timestamp, updated_at TIMESTAMP NOT NULL DEFAULT current_timestamp);
"""

# Append to query
sql_script_str = sql_script_str + create_records_table_query


# Iterate through all DF rows
for r in tqdm(range(n_rows)):
    # Add patient to patients table
    add_patient_query = f"""INSERT INTO patients(patient_id) VALUES('{df.iloc[r]['patient_id']}');"""
    sql_script_str = sql_script_str + add_patient_query + "\n"
    
    # Add record to records table
    add_record_query = """INSERT INTO records("""
    
    col_names = ""
    values = ""
    
    for c in range(n_cols):
        col_name = df.columns[c].replace(".", "")
        col_names = col_names + col_name + ", "
        
        value = df.iloc[r][str(df.columns[c])]
        
        if get_sql_datatype(str(df.dtypes[c])) == "VARCHAR(20)":
            values = values + "'" + str(value) + "', "
        else:
            values = values + str(value) + ", "
        
    col_names = col_names[:-2]
    values = values[:-2]
    
    add_record_query = add_record_query + col_names + ") VALUES (" + values + ");"
        
        #create_records_table_query = create_records_table_query + f"{col_name} {col_type}, "

    sql_script_str = sql_script_str + add_record_query + "\n"

#print(sql_script_str)

# Write script to file
with open("./data/init.sql", "w") as f:
    f.write(sql_script_str)

print("done")

100%|██████████| 497/497 [00:21<00:00, 23.06it/s]

done
